In [1]:
import numpy as np
import pandas as pd

In [38]:
%%time
df = pd.read_csv("yellow_tripdata_2016-06.csv", sep=',', dtype={"VendorID":np.int8,
                                                                            "passenger_count":np.int8,
                                                                            "trip_distance":float,
                                                                            "pickup_longitude":float,
                                                                            "pickup_latitude":float,
                                                                            "RatecodeID":np.int8,
                                                                            "store_and_fwd_flag":str,
                                                                            "dropoff_longitude":float,
                                                                            "dropoff_latitude":float,
                                                                            "payment_type":np.int8,
                                                                            "fare_amount":float,
                                                                            "extra":float,
                                                                            "mta_tax":float,
                                                                            "tip_amount":float,
                                                                            "tolls_amount":float,
                                                                            "improvement_surcharge":float,
                                                                            "total_amount":float})

Wall time: 8min 12s


In [39]:
df['store_and_fwd_flag'].replace(['N', 'Y'],[0, 1], inplace=True)
df['store_and_fwd_flag'] = df['store_and_fwd_flag'].astype(np.int8)

In [40]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"],format="%Y-%m-%d %H:%M:%S", exact=True)
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"],format="%Y-%m-%d %H:%M:%S")
df['trip_duration'] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).astype('timedelta64[m]')

In [41]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration
0,2,2016-06-09 21:06:36,2016-06-09 21:13:08,2,0.79,-73.983360,40.760937,1,0,-73.977463,40.753979,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30,6.0
1,2,2016-06-09 21:06:36,2016-06-09 21:35:11,1,5.22,-73.981720,40.736668,1,0,-73.981636,40.670242,1,22.0,0.5,0.5,4.00,0.0,0.3,27.30,28.0
2,2,2016-06-09 21:06:36,2016-06-09 21:13:10,1,1.26,-73.994316,40.751072,1,0,-74.004234,40.742168,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36,6.0
3,2,2016-06-09 21:06:36,2016-06-09 21:36:10,1,7.39,-73.982361,40.773891,1,0,-73.929466,40.851540,1,26.0,0.5,0.5,1.00,0.0,0.3,28.30,29.0
4,2,2016-06-09 21:06:36,2016-06-09 21:23:23,1,3.10,-73.987106,40.733173,1,0,-73.985909,40.766445,1,13.5,0.5,0.5,2.96,0.0,0.3,17.76,16.0


In [43]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11135470 entries, 0 to 11135469
Data columns (total 20 columns):
VendorID                 11135470 non-null int8
tpep_pickup_datetime     11135470 non-null datetime64[ns]
tpep_dropoff_datetime    11135470 non-null datetime64[ns]
passenger_count          11135470 non-null int8
trip_distance            11135470 non-null float64
pickup_longitude         11135470 non-null float64
pickup_latitude          11135470 non-null float64
RatecodeID               11135470 non-null int8
store_and_fwd_flag       11135470 non-null int8
dropoff_longitude        11135470 non-null float64
dropoff_latitude         11135470 non-null float64
payment_type             11135470 non-null int8
fare_amount              11135470 non-null float64
extra                    11135470 non-null float64
mta_tax                  11135470 non-null float64
tip_amount               11135470 non-null float64
tolls_amount             11135470 non-null float64
improvement_surchar

In [44]:
# make_structure_df - Функция создания пандас-таблицы, описывающей структуру данных 
# Параметры:
# make_col_df - создать таблицу (True) или просто вывести на печать краткую структуру данных (False)
# nan_count - показать количетсов пропущенных данных в столбце
# max_nuniques_for_print - максимальное количество уникальных значений для вывода в столбце "uniques"
# max_nuniques_as_categorial - максимальное количество уникальных значений в столбце, при котором тип столбца признаётся категориальным

def make_structure_df(data, make_col_df=True, nan_count=False, max_nuniques_for_print = 9, max_nuniques_as_categorial = 9):
    
    columns=['col_name', 'col_type', 'type', 'nunique', 'uniques',]
    
    if make_col_df == True:
        cols = []
    for col_name in data.columns:
        
        col_nunique = data[col_name].nunique()
        ctype = data[col_name].dtype.name
        
        if make_col_df == True:
            if (ctype == 'object') or (col_nunique <= max_nuniques_as_categorial):
                col_type = "categorial"
            elif ctype == 'datetime64[ns]':
                col_type = "datatime"
            else:
                col_type = "numeric"
            
            col_uniques = "..."
            if col_nunique <= max_nuniques_for_print:
                col_uniques = data[col_name].unique()
            cols.append([col_name, col_type, ctype, col_nunique, col_uniques])
    
    structure_df = pd.DataFrame(cols, columns=columns)
        
    if nan_count == True:
        structure_df['nan_count'] = data.isnull().sum(axis=0).tolist()
    
    return structure_df

In [45]:
df_structure = make_structure_df(df)
df_structure

,col_name,col_type,type,nunique,uniques
0,VendorID,categorial,int8,2,"[2, 1]"
1,tpep_pickup_datetime,datatime,datetime64[ns],2395442,...
2,tpep_dropoff_datetime,datatime,datetime64[ns],2399652,...
3,passenger_count,numeric,int8,10,...
4,trip_distance,numeric,float64,4581,...
5,pickup_longitude,numeric,float64,35135,...
6,pickup_latitude,numeric,float64,61876,...
7,RatecodeID,categorial,int8,7,"[1, 3, 2, 5, 4, 6, 99]"
8,store_and_fwd_flag,categorial,int8,2,"[0, 1]"
9,dropoff_longitude,numeric,float64,55408,...


In [47]:
df.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration
count,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07,1.113547e+07
mean,1.529817e+00,1.657273e+00,3.044006e+00,-7.305081e+01,4.024282e+01,1.043880e+00,6.046444e-03,-7.312388e+01,4.028391e+01,1.349718e+00,1.350708e+01,3.407188e-01,4.973046e-01,1.842121e+00,3.402089e-01,2.996818e-01,1.683016e+01,1.361669e+01
std,4.991102e-01,1.302489e+00,2.183019e+01,8.208047e+00,4.521673e+00,5.660610e-01,7.752345e-02,7.880313e+00,4.341196e+00,4.944984e-01,2.755358e+02,5.339716e-01,4.451916e-02,2.713585e+00,1.719710e+00,1.358086e-02,2.758608e+02,4.945371e+03
min,1.000000e+00,0.000000e+00,0.000000e+00,-1.181863e+02,0.000000e+00,1.000000e+00,0.000000e+00,-1.181863e+02,0.000000e+00,1.000000e+00,-4.500000e+02,-4.123000e+01,-2.700000e+00,-6.770000e+01,-1.250000e+01,-3.000000e-01,-4.508000e+02,-1.051915e+07
25%,1.000000e+00,1.000000e+00,1.000000e+00,-7.399178e+01,4.073653e+01,1.000000e+00,0.000000e+00,-7.399123e+01,4.073492e+01,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.800000e+00,6.000000e+00
50%,2.000000e+00,1.000000e+00,1.710000e+00,-7.398135e+01,4.075358e+01,1.000000e+00,0.000000e+00,-7.397935e+01,4.075412e+01,1.000000e+00,1.000000e+01,0.000000e+00,5.000000e-01,1.350000e+00,0.000000e+00,3.000000e-01,1.230000e+01,1.100000e+01
75%,2.000000e+00,2.000000e+00,3.230000e+00,-7.396617e+01,4.076831e+01,1.000000e+00,0.000000e+00,-7.396202e+01,4.076954e+01,2.000000e+00,1.550000e+01,5.000000e-01,5.000000e-01,2.460000e+00,0.000000e+00,3.000000e-01,1.836000e+01,1.900000e+01
max,2.000000e+00,9.000000e+00,7.173270e+04,0.000000e+00,6.409648e+01,9.900000e+01,1.000000e+00,1.062469e+02,6.004071e+01,5.000000e+00,6.285447e+05,5.979200e+02,6.035000e+01,8.548500e+02,9.700000e+02,1.164000e+01,6.290338e+05,3.780000e+03


In [46]:
df.to_pickle("df_task1.pkl")
#df = pd.read_pickle("df_task1.pkl")